In [50]:
import pandas as pd
import numpy as np
import random

df_fiscality = pd.read_csv('df_fiscality.csv')
df_MA_temp = pd.read_csv('df_MA_clean.csv')


In [51]:
df_MA_clean = df_MA_temp.drop(columns=['Taux_Global_TFB'])

In [52]:
mask = df_MA_clean['loyer_maison'].isna()

if mask.any(): 
    df_MA_clean.loc[mask, 'loyer_maison'] = df_MA_clean.loc[mask, 'loyer_appartement']
    df_MA_clean.loc[mask, 'loyer_appartement'] = np.nan
    
    df_MA_clean.loc[mask, 'loyer_min_maison'] = df_MA_clean.loc[mask, 'loyer_min_appartement']
    df_MA_clean.loc[mask, 'loyer_min_appartement'] = np.nan
    
    df_MA_clean.loc[mask, 'loyer_max_maison'] = df_MA_clean.loc[mask, 'loyer_max_appartement']
    df_MA_clean.loc[mask, 'loyer_max_appartement'] = np.nan


In [53]:
from datetime import datetime

today = datetime.today()
current_year = today.year
link_year = current_year - 1

# Filter for current year
df_current = df_fiscality[df_fiscality['EXERCICE'] == current_year]

if not df_current.empty:
    df_fiscality_2024 = df_current
else:
    # Fallback to previous year
    df_fiscality_2024 = df_fiscality[df_fiscality['EXERCICE'] == link_year]

# Select the needed columns
df_fiscality_clean = df_fiscality_2024[['Taux_Global_TFB', 'INSEE COM']]


In [54]:
# join DF on 'Code_insee' and 'INSEE COM'
df_MA_clean['Code_insee'] = df_MA_clean['Code_insee'].astype(str)
df_fiscality_clean['INSEE COM'] = df_fiscality_clean['INSEE COM'].astype(str)

df_merged = df_MA_clean.merge(df_fiscality_clean, how='left', left_on='Code_insee', right_on='INSEE COM')

C:\Users\vince\AppData\Local\Temp\ipykernel_9300\3420464112.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fiscality_clean['INSEE COM'] = df_fiscality_clean['INSEE COM'].astype(str)


In [55]:
df_merged_clean = df_merged[['ville','prix_appartement','min_appartement','max_appartement','prix_maison','min_maison','max_maison','loyer_appartement','loyer_min_appartement','loyer_max_appartement','loyer_maison','loyer_min_maison','loyer_max_maison','Taux_Global_TFB','Code_insee','Departement','geo']]
df_merged['Taux_Global_TFB'] = df_merged_clean['Taux_Global_TFB'] / 100

In [56]:
# example df for calculation
df_calcul = df_merged_clean.copy()

In [57]:
# nb fixe
df_calcul['nb_m2'] = 50
df_calcul['nb_mois'] = 12
df_calcul['assurance_pno_annuel'] = 300
df_calcul['frais_agence_%'] = 0.05
df_calcul['frais_notaire_%'] = 0.08
df_calcul['garantie_loyer_impaye'] = 0.03
df_calcul['compta_annuelle'] = 600
df_calcul['prevision_entretien_annuel'] = 2000
df_calcul['prevision_gros_oeuvre_annuel'] = 50000
df_calcul['taux_occupation'] = 0.95

# nb variable
df_calcul['loyer_mensuel_appart'] = df_calcul['loyer_appartement'] * df_calcul['nb_m2'] * df_calcul['taux_occupation']
df_calcul['loyer_mensuel_maison'] = df_calcul['loyer_maison'] * df_calcul['nb_m2'] * df_calcul['taux_occupation']
df_calcul['loyer_annuel_appart'] = df_calcul['loyer_mensuel_appart'] * df_calcul['nb_mois']
df_calcul['loyer_annuel_maison'] = df_calcul['loyer_mensuel_maison'] * df_calcul['nb_mois']
df_calcul['garantie_loyer_impaye_annuel_appart'] = df_calcul['garantie_loyer_impaye'] * df_calcul['loyer_annuel_appart']
df_calcul['garantie_loyer_impaye_annuel_maison'] = df_calcul['garantie_loyer_impaye'] * df_calcul['loyer_annuel_maison']
df_calcul['prix_net_vendeur_appart'] = df_calcul['prix_appartement'] * df_calcul['nb_m2']
df_calcul['prix_net_vendeur_maison'] = df_calcul['prix_maison'] * df_calcul['nb_m2']
df_calcul['frais_agence_maison'] = df_calcul['frais_agence_%'] * df_calcul['prix_net_vendeur_maison'] 
df_calcul['frais_agence_appart'] = df_calcul['frais_agence_%'] * df_calcul['prix_net_vendeur_appart'] 
df_calcul['frais_notaire_maison'] = df_calcul['frais_notaire_%'] * df_calcul['prix_net_vendeur_maison'] 
df_calcul['frais_notaire_appart'] = df_calcul['frais_notaire_%'] * df_calcul['prix_net_vendeur_appart'] 
df_calcul['prevision_entretien_annuel_appart'] = df_calcul['loyer_annuel_appart'] * 0.02
df_calcul['prevision_entretien_annuel_maison'] = df_calcul['loyer_annuel_maison'] * 0.02
df_calcul['prevision_gros_oeuvre_annuel_appart'] = df_calcul['prix_net_vendeur_appart']  * 0.005
df_calcul['prevision_gros_oeuvre_annuel_maison'] = df_calcul['prix_net_vendeur_maison'] * 0.005
df_calcul['valeur_cadastrale_locative_appart'] = df_calcul['loyer_annuel_appart'] / 2.5
df_calcul['valeur_cadastrale_locative_maison'] = df_calcul['loyer_annuel_maison'] / 2.5
df_calcul['TFPB_annuel_appart'] = df_calcul['valeur_cadastrale_locative_appart'] * df_calcul['Taux_Global_TFB'] / 100
df_calcul['TFPB_annuel_maison'] = df_calcul['valeur_cadastrale_locative_maison'] * df_calcul['Taux_Global_TFB'] / 100
df_calcul['charges_copro_annuel'] = 50
df_calcul['assurance_habitation_annuel'] = 0
df_calcul['frais_gestion_loc_annuel'] = 200
df_calcul['prix_achat_total_appart'] = df_calcul['prix_net_vendeur_appart'] + df_calcul['frais_agence_appart'] + df_calcul['frais_notaire_appart']
df_calcul['prix_achat_total_maison'] = df_calcul['prix_net_vendeur_maison'] + df_calcul['frais_agence_maison'] + df_calcul['frais_notaire_maison']
df_calcul['renta_brute_appart'] = df_calcul['loyer_annuel_appart'] / df_calcul['prix_achat_total_appart']
df_calcul['renta_brute_maison'] = df_calcul['loyer_annuel_maison'] / df_calcul['prix_achat_total_maison']
df_calcul['charges_annuelles_appart'] = df_calcul['assurance_pno_annuel'] + df_calcul['garantie_loyer_impaye_annuel_appart'] + df_calcul['TFPB_annuel_appart'] + df_calcul['charges_copro_annuel'] + df_calcul['assurance_habitation_annuel'] + df_calcul['frais_gestion_loc_annuel'] 
df_calcul['charges_annuelles_maison'] = df_calcul['assurance_pno_annuel'] + df_calcul['garantie_loyer_impaye_annuel_maison'] + df_calcul['TFPB_annuel_maison'] + df_calcul['charges_copro_annuel'] + df_calcul['assurance_habitation_annuel'] + df_calcul['frais_gestion_loc_annuel'] 
df_calcul['renta_nette_appart'] = (df_calcul['loyer_annuel_appart'] - df_calcul['charges_annuelles_appart']) / df_calcul['prix_achat_total_appart']
df_calcul['renta_nette_maison'] = (df_calcul['loyer_annuel_maison'] - df_calcul['charges_annuelles_maison']) / df_calcul['prix_achat_total_maison']

In [58]:
df_calcul.to_csv('example_df_MA_calcul.csv',index=False)

In [59]:
mask_dep44 = (df_merged_clean['Departement'] == 'Loire-Atlantique')
mask_dep85 = (df_merged_clean['Departement'] == 'Vendée')
mask_dep49 = (df_merged_clean['Departement'] == 'Maine-et-Loire')
mask_dep53 = (df_merged_clean['Departement'] == 'Mayenne')
mask_dep72 = (df_merged_clean['Departement'] == 'Sarthe')
 


In [60]:
if (df_merged_clean.loc[mask_dep44, 'Taux_Global_TFB'].isna().any()):
    mean_value = df_merged_clean.loc[mask_dep44, 'Taux_Global_TFB'].mean()
    df_merged_clean.loc[mask_dep44 & df_merged_clean['Taux_Global_TFB'].isna(), 'Taux_Global_TFB'] = mean_value

if (df_merged_clean.loc[mask_dep85, 'Taux_Global_TFB'].isna().any()):
    mean_value = df_merged_clean.loc[mask_dep85, 'Taux_Global_TFB'].mean()
    df_merged_clean.loc[mask_dep85 & df_merged_clean['Taux_Global_TFB'].isna(), 'Taux_Global_TFB'] = mean_value

if (df_merged_clean.loc[mask_dep49, 'Taux_Global_TFB'].isna().any()):
    mean_value = df_merged_clean.loc[mask_dep49, 'Taux_Global_TFB'].mean()
    df_merged_clean.loc[mask_dep49 & df_merged_clean['Taux_Global_TFB'].isna(), 'Taux_Global_TFB'] = mean_value

if (df_merged_clean.loc[mask_dep53, 'Taux_Global_TFB'].isna().any()):
    mean_value = df_merged_clean.loc[mask_dep53, 'Taux_Global_TFB'].mean()
    df_merged_clean.loc[mask_dep53 & df_merged_clean['Taux_Global_TFB'].isna(), 'Taux_Global_TFB'] = mean_value

if (df_merged_clean.loc[mask_dep72, 'Taux_Global_TFB'].isna().any()):
    mean_value = df_merged_clean.loc[mask_dep72, 'Taux_Global_TFB'].mean()
    df_merged_clean.loc[mask_dep72 & df_merged_clean['Taux_Global_TFB'].isna(), 'Taux_Global_TFB'] = mean_value


In [61]:
df_merged_clean.to_csv('df_MA_calcul.csv',index=False)